In [3]:
####################################
## A. Librerias
####################################
import os
import time
import json

import datetime
import pandas as pd

from utils.utils import *
from utils.paths import *

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

####################################
## B. Funciones
####################################

def get_files(path, extension):
    dir_name  = os.path.join(os.getcwd(), path)
    files     = os.listdir(dir_name)
    
    paths    = []
    for file in files:
        if extension in file:
            paths.append(os.path.join(path,file))

    return paths

def columns_to_string(df):
    columns = list(df.columns)    
    for column in columns:
        df[column] = df[column].astype(str)
    return df

####################################
## C. Configuración de parametros
####################################

HEADLESS = False
PRINT_VIEW = False
# CHROME_DRIVER_PATH = False

In [4]:
########################
## D. Ejecución
########################

# 1. Iniciar el driver, el servidor proxy y el proxy
driver, server, proxy = get_chrome_driver(chromedriver_path=CHROMEDRIVER_PATH, print_view=PRINT_VIEW, headless=HEADLESS)
print('1. Se inició el driver, el servidor proxy y el proxy')

Se inicia el servidor en Windows
Usando el chromedriver_path: C:\GitHubProjets\ScrapTrello\chromedriver\windows\chromedriver.exe


SessionNotCreatedException: Message: session not created: This version of ChromeDriver only supports Chrome version 114
Current browser version is 116.0.5845.112 with binary path C:\Program Files\Google\Chrome\Application\chrome.exe
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00C1A813+48355]
	(No symbol) [0x00BAC4B1]
	(No symbol) [0x00AB5358]
	(No symbol) [0x00AD61AC]
	(No symbol) [0x00AD1EF3]
	(No symbol) [0x00AD0579]
	(No symbol) [0x00B00C55]
	(No symbol) [0x00B0093C]
	(No symbol) [0x00AFA536]
	(No symbol) [0x00AD82DC]
	(No symbol) [0x00AD93DD]
	GetHandleVerifier [0x00E7AABD+2539405]
	GetHandleVerifier [0x00EBA78F+2800735]
	GetHandleVerifier [0x00EB456C+2775612]
	GetHandleVerifier [0x00CA51E0+616112]
	(No symbol) [0x00BB5F8C]
	(No symbol) [0x00BB2328]
	(No symbol) [0x00BB240B]
	(No symbol) [0x00BA4FF7]
	BaseThreadInitThunk [0x75CA7D59+25]
	RtlInitializeExceptionChain [0x77A7B79B+107]
	RtlClearBits [0x77A7B71F+191]


In [4]:
# 2. Eliminar todos los archivos de la carpeta
try:
    os.mkdir('jsons')
    print('Se creó la carpeta ./jsons')
except:
    print('Ya existe  ./jsons')

try:
    os.mkdir('jsons/trello')
    print('Se creó la carpeta ./jsons/trello')
except:
    print('Ya existe ./jsons/trello')

try:
    os.mkdir(f'jsons/trello')
    print(f'Se creó la carpeta ./jsons/trello')
except:
    print(f'Ya existe ./jsons/trello')

try:
    files = get_files(os.path.join(os.getcwd(),'jsons','trello'), '.')
    for file in files:
        os.remove(file)
    print('Se eliminaron los archivos de la carpeta')
except:
    pass

# 3. Iniciar .har
options = {
    'captureHeaders': True, # Capturar los headers --> Default: False
    'captureContent': True, # Capturar el contenido --> Default: False
    'captureBinaryContent': False, # Capturar el contenido binario --> Default: False
    'captureCookies': True, # Capturar las cookies --> Default: False
    'captureHeadersSize': -1, 
    'captureMaxSize': -1, 
    'captureBinaryContentMaxLength': -1 # Tamaño máximo del contenido binario
}
proxy.new_har("trello", options=options)

Se inicia el servidor en Windows
Usando el chromedriver_path: C:\GitHubProjets\ScrapTrello\chromedriver\windows\chromedriver.exe


SessionNotCreatedException: Message: session not created: This version of ChromeDriver only supports Chrome version 114
Current browser version is 116.0.5845.111 with binary path C:\Program Files\Google\Chrome\Application\chrome.exe
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00D4A813+48355]
	(No symbol) [0x00CDC4B1]
	(No symbol) [0x00BE5358]
	(No symbol) [0x00C061AC]
	(No symbol) [0x00C01EF3]
	(No symbol) [0x00C00579]
	(No symbol) [0x00C30C55]
	(No symbol) [0x00C3093C]
	(No symbol) [0x00C2A536]
	(No symbol) [0x00C082DC]
	(No symbol) [0x00C093DD]
	GetHandleVerifier [0x00FAAABD+2539405]
	GetHandleVerifier [0x00FEA78F+2800735]
	GetHandleVerifier [0x00FE456C+2775612]
	GetHandleVerifier [0x00DD51E0+616112]
	(No symbol) [0x00CE5F8C]
	(No symbol) [0x00CE2328]
	(No symbol) [0x00CE240B]
	(No symbol) [0x00CD4FF7]
	BaseThreadInitThunk [0x75DF7D59+25]
	RtlInitializeExceptionChain [0x7728B79B+107]
	RtlClearBits [0x7728B71F+191]
